In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import glob
import json
import numpy as np
import os
import shutil


import torch
from tqdm import tqdm
from pycocotools.coco import COCO  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb
import clip

from vis_prior.utils import *

/opt/conda/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
unfiltered_data_folder = "/media/data/dad/cnet/experiments/coco10novel/mix_n2000_o1_s1_p640_promptenhanced"
unfiltered_annotation_path = os.path.join(unfiltered_data_folder, "annotation.json")

In [4]:
with open(unfiltered_annotation_path, "r") as f:
    unfiltered_annotation = json.load(f)
coco = COCO(unfiltered_annotation_path)

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [6]:
device = "cuda"
model, preprocess = clip.load("ViT-L/14", device=device)

In [7]:
from PIL import Image
for ann in tqdm(unfiltered_annotation['annotations']):
    ann_id = ann["id"]
    x, y, w, h = ann['bbox']
    
    image_id = ann['image_id']
    img_obj = coco.loadImgs([image_id])[0]
    img_path = os.path.join(unfiltered_data_folder, 'images', img_obj['file_name'])
    img = Image.open(img_path)
    img = img.crop((x, y, x+w, y+h))
    #img = imread(img_path)
    
    cat_id = ann['category_id']
    cat_obj = coco.loadCats([cat_id])[0]
    cat_name = cat_obj["name"]
    
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize([cat_name, "a " + cat_name, "picture of a " + cat_name]).to(device)
    
    #import matplotlib.pyplot as plt
    #plt.imshow(img)
    #print([cat_name, "a " + cat_name, "picture of a " + cat_name])
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        
        #image_features /= image_features.norm(dim=-1, keepdim=True)
        #text_features /= text_features.norm(dim=-1, keepdim=True)
        #similarity = (100.0 * image_features @ text_features.T)

        logits_per_image, logits_per_text = model(image, text)
        #probs = logits_per_image.softmax(dim=-1).cpu().numpy()[0]
    
    #print(logits_per_image)
    #print(logits_per_image)
    #print(logits_per_text)
    #print(probs)
    
    ann["csl"] = float(logits_per_image[0][0]/100) # CLIP Similarity Score VIT-L14 (csl)    
    ann["csl_a"] = float(logits_per_image[0][1]/100) # a: "a xxxx"    
    ann["csl_p"] = float(logits_per_image[0][2]/100) # p: "a picture of xxx"

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2200/2200 [01:49<00:00, 20.13it/s]


In [8]:
print(len(unfiltered_annotation['annotations']))
print(unfiltered_annotation['annotations'][200])

2200
{'image_id': 570016, 'bbox': [9, 147, 103, 77], 'area': 7931.0, 'category_id': 9, 'id': 2223022, 'csl': 0.2283935546875, 'csl_a': 0.224365234375, 'csl_p': 0.2259521484375}


In [9]:
with open(unfiltered_annotation_path, "w+") as f:
    json.dump(unfiltered_annotation, f)